In [0]:
target_catalog = "newdbhive"

# Step 1: Get all Hive schemas
hive_schemas = [row.databaseName for row in spark.sql("SHOW DATABASES IN hive_metastore").collect()]

system_schemas = {"default", "information_schema"}

for hive_schema in hive_schemas:
    if hive_schema in system_schemas:
        continue

    target_schema = "{}.{}".format(target_catalog, hive_schema)
    print("\n=== Processing schema: {} → {} ===".format(hive_schema, target_schema))

    # Step 2: Create schema in UC if not exists
    spark.sql("CREATE SCHEMA IF NOT EXISTS {}".format(target_schema))

    # Step 3: Fetch UC table list once per schema (fix for duplicate creation)
    uc_existing_tables = [r.tableName for r in spark.sql("SHOW TABLES IN {}".format(target_schema)).collect()]

    # Step 4: List Hive tables
    hive_tables = [row.tableName for row in spark.sql(
        "SHOW TABLES IN hive_metastore.{}".format(hive_schema)
    ).collect()]

    for table in hive_tables:
        src = "hive_metastore.{}.{}".format(hive_schema, table)
        dest = "{}.{}".format(target_schema, table)

        print("→ Checking table: {} → {}".format(src, dest))

        if table not in uc_existing_tables:
            print("   Creating new UC table {}".format(dest))
            spark.sql("CREATE TABLE {} AS SELECT * FROM {}".format(dest, src))
            # Add to existing table list to prevent re-creation in same run
            uc_existing_tables.append(table)
        else:
            print("   UC table {} already exists — appending data".format(dest))
            src_df = spark.table(src)
            src_df.write.mode("append").saveAsTable(dest)

print("\n✅ Migration completed successfully!")